In [ ]:
import fdapdepy as fdapde
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation

In [ ]:
# mesh
nodes = np.loadtxt("unit_square/points.txt", dtype=float)
triangles = np.loadtxt("unit_square/elements.txt", dtype=int)
triangles = triangles - 1 # sempre stessa storia
boundary = np.loadtxt("unit_square/boundary.txt", dtype=int)
nodes.shape

mesh = fdapde.domain(nodes, triangles, boundary)

type(mesh)

In [ ]:
triangulation = Triangulation(mesh.nodes()[:,0], mesh.nodes()[:,1], mesh.elements())

fig, ax = plt.subplots()
ax.triplot(triangulation)

In [ ]:
basis = fdapde.functional_basis(mesh, 1)
basis.size()
# non parametric comp
f = fdapde.function(basis)

In [ ]:
Lf = -fdapde.laplacian(f)  
penalty = fdapde.pde(Lf) # forcing == 0.

In [ ]:
u_ex = np.sin(np.pi*mesh.nodes()[:,0]) * np.sin(np.pi*mesh.nodes()[:,1])
u_ex = u_ex.reshape((u_ex.shape[0],1))
obs = u_ex + 0.05 * np.random.randn(nodes.shape[0],1)

In [ ]:
model = fdapde.srpde(y=obs, data=None, penalty=penalty)

gcv_ = fdapde.gcv(lambda_= np.array([1e-5, 1e-4,1e-3, 1e-2, 1e-1, 1]).reshape(6,1))

In [ ]:
gcv_

In [ ]:
model.fit(calibration = gcv_)

In [ ]:
print(model.optimal_lambda())
print(model.f().shape)
print(u_ex.shape)
def rmse(x,y):
    return np.sqrt( np.mean( (x-y)*(x-y) ))
    
print("err_l_inf = ", np.max(np.abs(model.f() - u_ex.reshape(-1))))
print("err_l_2 = ", rmse( model.f(), u_ex.reshape(-1)))

In [ ]:
#plt.figure(figsize=(20,12))
#plt.style.use('_mpl-gallery-nogrid')

fig, ax = plt.subplots(1,2)
#ax.plot(nodes[:,0], nodes[:,1], markersize=, color='grey')
ax[0].tripcolor(nodes[:,0], nodes[:,1], u_ex.reshape(-1))
ax[0].set(xlim=(0, 1), ylim=(0, 1))

ax[1].tripcolor(nodes[:,0], nodes[:,1], model.f().reshape(-1))
ax[1].set(xlim=(0, 1), ylim=(0, 1))
plt.show()

In [ ]:
fig, ax = plt.subplots(1,2)
#ax.plot(nodes[:,0], nodes[:,1], markersize=, color='grey')
ax[0].tricontourf(nodes[:,0], nodes[:,1], u_ex.reshape(-1))
ax[0].set(xlim=(0, 1), ylim=(0, 1))

ax[1].tricontourf(nodes[:,0], nodes[:,1], model.f().reshape(-1))
ax[1].set(xlim=(0, 1), ylim=(0, 1))
plt.show()

In [ ]:
### --- formula 
import pandas as pd
data = pd.DataFrame(obs, columns=["y"])

In [ ]:
model = fdapde.srpde(formula="y ~ f", data=data, 
                     penalty=penalty)
gcv_ = fdapde.gcv(lambda_= np.array([1e-5, 1e-4,1e-3, 1e-2, 1e-1, 1]).reshape(6,1))
model.fit(calibration = gcv_)
print("err_l_inf = ", np.max(np.abs(model.f() - u_ex.reshape(-1))))
print("err_l_2 = ", rmse( model.f(), u_ex.reshape(-1)))

fig, ax = plt.subplots(1,2)
ax[0].tripcolor(nodes[:,0], nodes[:,1], u_ex.reshape(-1))
ax[0].set(xlim=(0, 1), ylim=(0, 1))

ax[1].tripcolor(nodes[:,0], nodes[:,1], model.f().reshape(-1))
ax[1].set(xlim=(0, 1), ylim=(0, 1))
plt.show()